In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def resample_raster(src_path, dst_path, reference_raster):
    with rasterio.open(src_path) as src:
        with rasterio.open(reference_raster) as ref:
            # Define target CRS and dimensions based on the reference raster
            dst_crs = ref.crs
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            
            # Prepare kwargs for the output raster
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': dst_transform,
                'width': dst_width,
                'height': dst_height
            })

            # Perform the reprojection and resampling
            with rasterio.open(dst_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=dst_transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.bilinear
                    )

# Paths to your Sentinel-1 and Sentinel-2 images
sentinel1_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1\S1A_IW_SLC__1SDV_20230314T005406_geo_10m.tif"
sentinel2_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2\reshaped_1calibrated_subset_0_of_S2B_MSIL2A_20230104T053219_N0509_R105_T43QEE_20230104T070636_resampled.tif"

# Output paths for resampled images
sentinel1_resampled_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1\resampled_S1A_IW_SLC__1SDV_20230314T005406_geo_10m.tif"

# Resample Sentinel-1 to match Sentinel-2
resample_raster(sentinel1_path, sentinel1_resampled_path, sentinel2_path)

print("Resampling complete.")


Resampling complete.


In [4]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject, transform_bounds
import numpy as np

def co_register_images(src_path, dst_path, reference_raster_path):
    with rasterio.open(reference_raster_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height
        
        with rasterio.open(src_path) as src:
            src_crs = src.crs
            src_transform = src.transform
            
            # Calculate the bounds of the source image in the reference CRS
            src_bounds = transform_bounds(src_crs, ref_crs, *src.bounds)
            
            # Calculate the transform and dimensions of the destination raster
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs, ref.crs, ref_width, ref_height, *src_bounds, resolution=(ref_transform[0], -ref_transform[4]))
            
            # Update metadata for the destination raster
            dst_meta = src.meta.copy()
            dst_meta.update({
                'crs': ref_crs,
                'transform': dst_transform,
                'width': dst_width,
                'height': dst_height
            })
            
            # Perform the resampling and reprojection
            with rasterio.open(dst_path, 'w', **dst_meta) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=dst_transform,
                        dst_crs=ref.crs,
                        resampling=Resampling.bilinear
                    )

# Paths to your Sentinel-1 and Sentinel-2 images
sentinel1_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1\resampled_S1A_IW_SLC__1SDV_20230101T005408_geo_10m.tif"
sentinel2_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2\resampled_subset_0_of_S2A_MSIL2A_20230310T052641_N0509_R105_T43QEE_20230310T091359_resampled.tif"

# Output path for the co-registered Sentinel-2 image
sentinel2_coregistered_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2\coregistered_subset_0_of_S2A_MSIL2A_20230310T052641_N0509_R105_T43QEE_20230310T091359_resampled.tif"
# Co-register Sentinel-2 to match Sentinel-1
co_register_images(sentinel2_path, sentinel2_coregistered_path, sentinel1_path)

print("Co-registration complete.")


Co-registration complete.
